# With rgb images

### Load data

In [ ]:
import pickle

train_filename = "D:/Minor/minor chester/train_data_sample_rgb.p"
(train_labels, train_data, train_tensors) = pickle.load(open(train_filename, mode='rb'))

valid_filename = "D:/Minor/minor chester/valid_data_sample_rgb.p"
(valid_labels, valid_data, valid_tensors) = pickle.load(open(valid_filename, mode='rb'))

test_filename = "'D:/Minor/minor chester/test_data_sample_rgb.p"
(test_labels, test_data, test_tensors) = pickle.load(open(test_filename, mode='rb'))

### CNN model

In [2]:
from keras import backend as K

def binary_accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred)))

def precision_threshold(threshold = 0.5):
    def precision(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(y_pred)
        precision_ratio = true_positives / (predicted_positives + K.epsilon())
        return precision_ratio
    return precision

def recall_threshold(threshold = 0.5):
    def recall(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.clip(y_true, 0, 1))
        recall_ratio = true_positives / (possible_positives + K.epsilon())
        return recall_ratio
    return recall

def fbeta_score_threshold(beta = 1, threshold = 0.5):
    def fbeta_score(y_true, y_pred):
        threshold_value = threshold
        beta_value = beta
        p = precision_threshold(threshold_value)(y_true, y_pred)
        r = recall_threshold(threshold_value)(y_true, y_pred)
        bb = beta_value ** 2
        fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
        return fbeta_score
    return fbeta_score

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "c:\users\arpit\appdata\local\programs\python\python37-32\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 18, in swig_import_helper
    fp, pathname, description = imp.find_module('_pywrap_tensorflow', [dirname(__file__)])
  File "c:\users\arpit\appdata\local\programs\python\python37-32\lib\imp.py", line 296, in find_module
    raise ImportError(_ERR_MSG.format(name), name=name)
ImportError: No module named '_pywrap_tensorflow'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\arpit\appdata\local\programs\python\python37-32\lib\site-packages\tensorflow\python\__init__.py", line 66, in <module>
    from tensorflow.python import pywrap_tensorflow
  File "c:\users\arpit\appdata\local\programs\python\python37-32\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 28, in <module>
    _pywrap_tensorflow = swig_import_helper()
  File "c:\us

In [8]:
import time

from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras import regularizers, initializers, optimizers

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=7,padding='same',activation='relu', 
                 input_shape=train_tensors.shape[1:]))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, 
                 kernel_size=5,
                 padding='same', 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, 
                 kernel_size=5,
                 padding='same', 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=128, 
                 kernel_size=5,
                 strides=2,
                 padding='same', 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 16)        2368      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        12832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 4, 128)         2

In [3]:
from keras.optimizers import SGD
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(optimizer='sgd', loss='binary_crossentropy', 
              metrics=[precision_threshold(threshold = 0.5), 
                       recall_threshold(threshold = 0.5), 
                       fbeta_score_threshold(beta=0.5, threshold = 0.5),
                      'accuracy'])

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "c:\users\arpit\appdata\local\programs\python\python37-32\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 18, in swig_import_helper
    fp, pathname, description = imp.find_module('_pywrap_tensorflow', [dirname(__file__)])
  File "c:\users\arpit\appdata\local\programs\python\python37-32\lib\imp.py", line 296, in find_module
    raise ImportError(_ERR_MSG.format(name), name=name)
ImportError: No module named '_pywrap_tensorflow'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\arpit\appdata\local\programs\python\python37-32\lib\site-packages\tensorflow\python\__init__.py", line 66, in <module>
    from tensorflow.python import pywrap_tensorflow
  File "c:\users\arpit\appdata\local\programs\python\python37-32\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 28, in <module>
    _pywrap_tensorflow = swig_import_helper()
  File "c:\us

In [29]:
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping
import numpy as np

epochs = 20
batch_size = 32

earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, mode='auto')
log = CSVLogger('C:/Users/behl/Desktop/minor/log_bCNN_rgb.csv')
checkpointer = ModelCheckpoint(filepath='C:/Users/behl/Desktop/minor/bCNN.best.from_scratch.hdf5', 
                               monitor = 'val_loss',verbose=1, save_best_only=True)

start = time.time()

model.fit(train_tensors, train_labels, 
          validation_data=(valid_tensors, valid_labels),
          epochs=epochs, batch_size=batch_size, callbacks=[checkpointer, log, earlystop], verbose=1)

# Show total training time
print("training time: %.2f minutes"%((time.time()-start)/60))

Train on 3400 samples, validate on 1100 samples
Epoch 1/20
3392/3400 [============================>.] - ETA: 0s - loss: 0.6840 - precision: 0.0000e+00 - recall: 0.0000e+00 - fbeta_score: 0.0000e+00 - acc: 0.5486
Epoch 00001: val_loss improved from inf to 0.68587, saving model to C:/Users/behl/Desktop/minor/bCNN.best.from_scratch.hdf5
3400/3400 [==============================] - 3s 776us/sample - loss: 0.6840 - precision: 0.0000e+00 - recall: 0.0000e+00 - fbeta_score: 0.0000e+00 - acc: 0.5485 - val_loss: 0.6859 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_fbeta_score: 0.0000e+00 - val_acc: 0.5345
Epoch 2/20
3328/3400 [============================>.] - ETA: 0s - loss: 0.6828 - precision: 0.0000e+00 - recall: 0.0000e+00 - fbeta_score: 0.0000e+00 - acc: 0.5478
Epoch 00002: val_loss improved from 0.68587 to 0.68453, saving model to C:/Users/behl/Desktop/minor/bCNN.best.from_scratch.hdf5
3400/3400 [==============================] - 3s 770us/sample - loss: 0.6827 - precision: 0.

Epoch 17/20
3392/3400 [============================>.] - ETA: 0s - loss: 0.6507 - precision: 0.6321 - recall: 0.4735 - fbeta_score: 0.5766 - acc: 0.6315
Epoch 00017: val_loss did not improve from 0.65567
3400/3400 [==============================] - 3s 781us/sample - loss: 0.6506 - precision: 0.6293 - recall: 0.4784 - fbeta_score: 0.5748 - acc: 0.6318 - val_loss: 0.6894 - val_precision: 0.6871 - val_recall: 0.1963 - val_fbeta_score: 0.4297 - val_acc: 0.5827
Epoch 18/20
3328/3400 [============================>.] - ETA: 0s - loss: 0.6481 - precision: 0.6332 - recall: 0.4561 - fbeta_score: 0.5669 - acc: 0.6262 ETA: 1s - loss: 0.6461 - precision: 0.6353 - recall: 0.4371
Epoch 00018: val_loss improved from 0.65567 to 0.65291, saving model to C:/Users/behl/Desktop/minor/bCNN.best.from_scratch.hdf5
3400/3400 [==============================] - 3s 837us/sample - loss: 0.6475 - precision: 0.6326 - recall: 0.4597 - fbeta_score: 0.5675 - acc: 0.6268 - val_loss: 0.6529 - val_precision: 0.6272 - val_

### Metric

In [4]:
model.load_weights('D:/Minor/minor chester/bCNN.best.from_scratch.hdf5')
prediction = model.predict(test_tensors)
print('prediction',prediction)

NameError: name 'model' is not defined

In [5]:
threshold = 0.5
beta = 0.5

pre = K.eval(precision_threshold(threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))
rec = K.eval(recall_threshold(threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))
fsc = K.eval(fbeta_score_threshold(beta = beta, threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))

print ("Precision: %f %%\nRecall: %f %%\nFscore: %f %%"% (pre, rec, fsc))

NameError: name 'K' is not defined

In [6]:
K.eval(binary_accuracy(K.variable(value=test_labels),
                                   K.variable(value=prediction)))

NameError: name 'K' is not defined

In [33]:
prediction[:30]

array([[0.4363243 ],
       [0.34560972],
       [0.6282389 ],
       [0.3624897 ],
       [0.44472387],
       [0.4790499 ],
       [0.60046285],
       [0.28872514],
       [0.31944084],
       [0.31732532],
       [0.34682706],
       [0.50103843],
       [0.6649924 ],
       [0.58886546],
       [0.5946922 ],
       [0.25947493],
       [0.6989844 ],
       [0.28664488],
       [0.6489586 ],
       [0.4390859 ],
       [0.42488033],
       [0.42834905],
       [0.43239993],
       [0.6806985 ],
       [0.53251773],
       [0.42801526],
       [0.4854939 ],
       [0.59501547],
       [0.26679447],
       [0.63957006]], dtype=float32)

In [34]:
threshold = 0.4
beta = 0.5

pre = K.eval(precision_threshold(threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))
rec = K.eval(recall_threshold(threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))
fsc = K.eval(fbeta_score_threshold(beta = beta, threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))

print ("Precision: %f %%\nRecall: %f %%\nFscore: %f %%"% (pre, rec, fsc))

Precision: 0.470588 %
Recall: 0.775510 %
Fscore: 0.510753 %


In [35]:
threshold = 0.6
beta = 0.5

pre = K.eval(precision_threshold(threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))
rec = K.eval(recall_threshold(threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))
fsc = K.eval(fbeta_score_threshold(beta = beta, threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))

print ("Precision: %f %%\nRecall: %f %%\nFscore: %f %%"% (pre, rec, fsc))

Precision: 0.634146 %
Recall: 0.265306 %
Fscore: 0.496183 %


# With gray images

In [36]:
import pickle

train_filename = "C:/Users/behl/Desktop/minor/train_data_sample_gray.p"
(train_labels, train_data, train_tensors) = pickle.load(open(train_filename, mode='rb'))

valid_filename = "C:/Users/behl/Desktop/minor/valid_data_sample_gray.p"
(valid_labels, valid_data, valid_tensors) = pickle.load(open(valid_filename, mode='rb'))

test_filename = "C:/Users/behl/Desktop/minor/test_data_sample_gray.p"
(test_labels, test_data, test_tensors) = pickle.load(open(test_filename, mode='rb'))

In [37]:
import time

from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras import regularizers, initializers, optimizers

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=7,padding='same',activation='relu', 
                 input_shape=train_tensors.shape[1:]))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, 
                 kernel_size=5,
                 padding='same', 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, 
                 kernel_size=5,
                 padding='same', 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=128, 
                 kernel_size=5,
                 strides=2,
                 padding='same', 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 64, 64, 16)        800       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 32, 32, 32)        12832     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 16, 16, 64)        51264     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 4, 4, 128)        

In [38]:
from keras import backend as K

def binary_accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred)))

def precision_threshold(threshold = 0.5):
    def precision(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(y_pred)
        precision_ratio = true_positives / (predicted_positives + K.epsilon())
        return precision_ratio
    return precision

def recall_threshold(threshold = 0.5):
    def recall(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.clip(y_true, 0, 1))
        recall_ratio = true_positives / (possible_positives + K.epsilon())
        return recall_ratio
    return recall

def fbeta_score_threshold(beta = 1, threshold = 0.5):
    def fbeta_score(y_true, y_pred):
        threshold_value = threshold
        beta_value = beta
        p = precision_threshold(threshold_value)(y_true, y_pred)
        r = recall_threshold(threshold_value)(y_true, y_pred)
        bb = beta_value ** 2
        fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
        return fbeta_score
    return fbeta_score

In [39]:
model.compile(optimizer='sgd', loss='binary_crossentropy', 
              metrics=[precision_threshold(threshold = 0.5), 
                       recall_threshold(threshold = 0.5), 
                       fbeta_score_threshold(beta=0.5, threshold = 0.5),
                      'accuracy'])

In [40]:
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping
import numpy as np

epochs = 20
batch_size = 32

earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, mode='auto')
log = CSVLogger('C:/Users/behl/Desktop/minor/log_bCNN_gray.csv')
checkpointer = ModelCheckpoint(filepath='C:/Users/behl/Desktop/minor/bCNN_gray.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

start = time.time()

model.fit(train_tensors, train_labels, 
          validation_data=(valid_tensors, valid_labels),
          epochs=epochs, batch_size=batch_size, callbacks=[checkpointer, log, earlystop], verbose=1)

# Show total training time
print("training time: %.2f minutes"%((time.time()-start)/60))

Train on 3400 samples, validate on 1100 samples
Epoch 1/20
3392/3400 [============================>.] - ETA: 0s - loss: 0.6862 - precision: 0.0000e+00 - recall: 0.0000e+00 - fbeta_score: 0.0000e+00 - acc: 0.5483
Epoch 00001: val_loss improved from inf to 0.68914, saving model to C:/Users/behl/Desktop/minor/bCNN_gray.best.from_scratch.hdf5
3400/3400 [==============================] - 4s 1ms/sample - loss: 0.6861 - precision: 0.0000e+00 - recall: 0.0000e+00 - fbeta_score: 0.0000e+00 - acc: 0.5485 - val_loss: 0.6891 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_fbeta_score: 0.0000e+00 - val_acc: 0.5345
Epoch 2/20
3360/3400 [============================>.] - ETA: 0s - loss: 0.6847 - precision: 0.0000e+00 - recall: 0.0000e+00 - fbeta_score: 0.0000e+00 - acc: 0.5485
Epoch 00002: val_loss improved from 0.68914 to 0.68617, saving model to C:/Users/behl/Desktop/minor/bCNN_gray.best.from_scratch.hdf5
3400/3400 [==============================] - 3s 793us/sample - loss: 0.6847 - preci

In [41]:
model.load_weights('C:/Users/behl/Desktop/minor/bCNN_gray.best.from_scratch.hdf5')
prediction = model.predict(test_tensors)
print('prediction',prediction)

prediction [[0.40149194]
 [0.35671026]
 [0.5295818 ]
 [0.34850624]
 [0.39328128]
 [0.37793422]
 [0.50623745]
 [0.3037331 ]
 [0.34244263]
 [0.3514979 ]
 [0.3343041 ]
 [0.41950437]
 [0.5871829 ]
 [0.52777535]
 [0.50695014]
 [0.26181662]
 [0.60289854]
 [0.2928931 ]
 [0.5821857 ]
 [0.42113817]
 [0.3810205 ]
 [0.4062845 ]
 [0.39560556]
 [0.58394516]
 [0.4489488 ]
 [0.4220333 ]
 [0.42086747]
 [0.51065487]
 [0.3628356 ]
 [0.51080847]
 [0.29644528]
 [0.28684705]
 [0.37411666]
 [0.44644073]
 [0.48257244]
 [0.562293  ]
 [0.38482314]
 [0.3003142 ]
 [0.29670328]
 [0.32793838]
 [0.5231838 ]
 [0.40151122]
 [0.32805848]
 [0.34543204]
 [0.3906322 ]
 [0.53488   ]
 [0.5029401 ]
 [0.577196  ]
 [0.35210353]
 [0.36534774]
 [0.3426104 ]
 [0.3797388 ]
 [0.38672924]
 [0.6194257 ]
 [0.48896292]
 [0.41161206]
 [0.4479072 ]
 [0.4495389 ]
 [0.37746477]
 [0.3737647 ]
 [0.49621266]
 [0.53151995]
 [0.42121005]
 [0.36705267]
 [0.38133258]
 [0.5202369 ]
 [0.54733145]
 [0.44987062]
 [0.27751392]
 [0.2847425 ]
 [0.38998

In [42]:
threshold = 0.5
beta = 0.5

pre = K.eval(precision_threshold(threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))
rec = K.eval(recall_threshold(threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))
fsc = K.eval(fbeta_score_threshold(beta = beta, threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))

print ("Precision: %f %%\nRecall: %f %%\nFscore: %f %%"% (pre, rec, fsc))

Precision: 0.594059 %
Recall: 0.306122 %
Fscore: 0.500000 %


In [43]:
K.eval(binary_accuracy(K.variable(value=test_labels),
                                   K.variable(value=prediction)))

0.64457834

In [44]:
prediction[:30]

array([[0.40149194],
       [0.35671026],
       [0.5295818 ],
       [0.34850624],
       [0.39328128],
       [0.37793422],
       [0.50623745],
       [0.3037331 ],
       [0.34244263],
       [0.3514979 ],
       [0.3343041 ],
       [0.41950437],
       [0.5871829 ],
       [0.52777535],
       [0.50695014],
       [0.26181662],
       [0.60289854],
       [0.2928931 ],
       [0.5821857 ],
       [0.42113817],
       [0.3810205 ],
       [0.4062845 ],
       [0.39560556],
       [0.58394516],
       [0.4489488 ],
       [0.4220333 ],
       [0.42086747],
       [0.51065487],
       [0.3628356 ],
       [0.51080847]], dtype=float32)

In [45]:
threshold = 0.4
beta = 0.5

pre = K.eval(precision_threshold(threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))
rec = K.eval(recall_threshold(threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))
fsc = K.eval(fbeta_score_threshold(beta = beta, threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))

print ("Precision: %f %%\nRecall: %f %%\nFscore: %f %%"% (pre, rec, fsc))

Precision: 0.509294 %
Recall: 0.698980 %
Fscore: 0.538522 %
